**Autor**: Fabián Felipe Quevedo Farieta 

Análisis Numérico 2022-2

## Problema 1

Considere el método de Shamanskii de orden $m$ definido de la siguiente manera. Dado $x_n$ calcule $x_{n+1}$ usando $m$ pasos del método de la cuerda, esto es, 

$z_1=x_n-f(x_n)/f'(x_n)$

$z_{j+1}=z_{j}-f(z_j)/f'(x_n)$, $1\leq j\leq m-1$

$x_{n+1}=z_{m}$.

Observe que se avanza m pasos sin actualizar la derivada en el punto. Implemente el método para un order $m$ definido por el usuario y verifique su código con una ecuación escalar. 

### Solución

In [2]:
function Shamanskii(f,fp,x0,eps,del,M,N)

   #= f : función
    fp : Derivada
    x0 : Valor inicial<
    epsilon : Tolerancia residuo.
    M : Número maximo de iteraciones z
    N : Número maximo de iteraciones x
    =#
    
    v=f(x0)
    w=fp(x0)
    x=x0
    n=1
    if abs(v)<eps
        print("residuo f(x(",n,"))=",v,"\n")
        return x
    end
    while n<=N
        if w==0
            print("Derivada nula, error!")
            return None
        end
        z1=x-v/w
        m=1
        while m<=M-1
            v=f(z1)
            z=z1-v/w
            z1=z
            m=m+1
            x=z
        end
        v=f(x)
        w=fp(x)
        if abs(x-x0)<del 
            print("incremento delta(",n,")=",v,"\n")
            return x
        end
        x0=x
        if abs(v)<eps
            print("residuo f(x(",n,",",M,"))=",v,"\n")
            return x
        end
        n=n+1
    end
    if n==N
        print("Número maximo de iteraciones")
    end 
    return x0
end

Shamanskii (generic function with 1 method)

In [3]:
function f(x)
    return exp(x)-sin(x)
end
function fp(x)
    return exp(x)-cos(x)
end
Shamanskii(f,fp,1.0,1e-10,1e-10,5,5) 
Shamanskii(f,fp,1.0,1e-10,1e-10,52,1)

residuo f(x(4,5))=2.1510571102112408e-16
residuo f(x(1,52))=8.940764795184464e-14


-3.1830630119332777

Del funcionamiento de la implementación es posible observar que con 4 iteraciones en N y 5 de M, es decir únicamente llegando a $x_2$ se logra un resultado de la raíz con un residuo de $10^{-16}$, similar que se lograba para la misma función con 10 iteraciones de Newton (veáse cuadernillo de la clase); también es destacable el hecho de que Newton convergió a la segunda raíz de la función y Shamanskii a la segunda. A pesar de que en total se hacen 20 iteraciones, el doble que en Newton, se debe tener en cuenta que el método de Shamanskii no actualiza la derivada en cada valor de z, únicamente lo hace para x, por lo que en un caso donde obetener la derivada sea computacionalmente más costosa que evaluar la función este método será muy útil.

Notese que si no actualizamos ni una vez la derivada, es decir, 1 iteración en N, se necesita usar el orden 52 de Shamanskii para obtener el mismo orden de residuo que con Newton; a pesar de que en este caso se quintuplican las iteraciones, se puede pensar que al tener como parámetros M y N se puede buscar la mejor combinación de ambas para buscar el error deseado.

## Problema 2

Resuelva las siguientes ecuaiones $f(x)=0$ usando el Método de newton, el método de la secante (para este inicie la iteración en $x_{-1}$ con $x_{-1}=0.99x_0$) y el método de Shamanskii con $m=2,3,4$. Comente los resultados. Puede hacer tablas o plots con las iteraciones.


1.   $f(x)=\cos(x)-x$,  $\quad x_0=.5$.
2.   $f(x)=\arctan(x)$,  $\quad x_0=1$.
3.   $f(x)=\sin(x)$, $\quad  x_0=3$.
4.   $f(x)=x^2$,  $\quad  x_0=.5$.
5.   $f(x)=x^2+1$,  $\quad  x_0=10$.


### Solución 

In [69]:
#implementemos Newton a partir de la funcion del cuadernillo
function minewton(f,fp,x0,myeps,mydel,max_iter)
    #=f(x)=0 usando newton xn=xa-f(xa)/fp(xa)
    ----------
    f : funnción
    fp : Derivada
    x0 : Aproximación inicial
    epsilon : Tolerancia residuo.
    M : Número maximo de iteraciones
    ----------
    =#
    
    #Vectores de ayuda para tablas
    X=zeros(N+1)
    Y=zeros(N+1)
    
    x = x0
    fx = f(x)
    X[1]=0
    Y[1]=abs(fx)
    n=1
    
    if abs(fx) < myeps
        return x
    end
    while n<=max_iter
        fpx = fp(x)
        if fpx == 0
            print("Derivada nula. Error!")
            return None
        end
        x = x - fx/fpx
        fx = f(x)
        X[n+1]=n
        Y[n+1]=abs(fx)
        if abs(x-x0) < mydel || abs(fx) < myeps
            return x,X,Y
        end
        x0=x
        n=n+1
    end
    return x0,X,Y
end

minewton (generic function with 1 method)

In [68]:
#implementemos el método de la secante
function secante(f,x0,eps,del,max_iter)
    #=    
    ----------
    f : funnción
    x0 : Aproximación inicial
    epsilon : Tolerancia residuo.
    N : Número maximo de iteraciones
    ----------
    =#
    
    #Vectores de ayuda para tablas
    X=zeros(N+1)
    Y=zeros(N+1)
    
    v=f(x0)
    xans=0.99*x0
    w=f(xans)
    x = x0
    X[1]=0
    Y[1]=abs(v)
    if abs(v)<del
       return x 
    end
    n=1
    while n<=max_iter
        x=x0-(v*(x0-xans)/(v-w))
        w=v
        v=f(x)
        X[n+1]=n
        Y[n+1]=abs(v)
        if abs(x-x0)<eps || abs(v)<del
            return x,X,Y
        end
        xans=x0
        x0=x
        n=n+1
    end
    return x,X,Y
end

secante (generic function with 1 method)

In [70]:
#Reimplementación de Shamanskii de tal forma que lo imprime de manera adecuada para el análisis
function Shamanskii2(f,fp,x0,eps,del,M,N)

   #= f : función
    fp : Derivada
    x0 : Valor inicial
    epsilon : Tolerancia residuo.
    M : Número maximo de iteraciones z
    N : Número maximo de iteraciones x
    =#
    
    #Vectores de ayuda para tablas
    X=zeros(N+1)
    Y=zeros(N+1)
    
    v=f(x0)
    w=fp(x0)
    x=x0
    X[1]=0
    Y[1]=abs(v)
    n=1
    if abs(v)<eps
        return x
    end
    while n<=N
        if w==0
            print("Derivada nula, error!")
            return None
        end
        z1=x-v/w
        m=1
        while m<=M-1
            v=f(z1)
            z=z1-v/w
            z1=z
            m=m+1
            x=z
        end
        v=f(x)
        w=fp(x)
        X[n+1]=n
        Y[n+1]=abs(v)
        if abs(x-x0)<del || abs(v)<eps
            return x,X,Y
        end
        x0=x
        n=n+1
    end
    if n==N
        print("Número maximo de iteraciones")
    end 
    return x0,X,Y
end

Shamanskii2 (generic function with 1 method)

In [42]:
using DataFrames

In [8]:
#functions
function f1(x)
    return cos(x)-x
end
function f2(x)
    return atan(x)
end
function f3(x)
    return sin(x)
end
function f4(x)
    return x^2
end
function f5(x)
    return (x^2)+1
end
#derivatives
function f1p(x)
    return -sin(x)-1
end
function f2p(x)
    return 1/(1+x^2)
end
function f3p(x)
    return cos(x)
end
function f4p(x)
    return 2*x
end
function f5p(x)
    return 2*x
end

f5p (generic function with 1 method)

In [95]:
x0=0.5
eps=1e-10
del=1e-10
N=6
x11,X1,Y1=Shamanskii2(f1,f1p,x0,eps,del,2,N)
x12,X2,Y2=Shamanskii2(f1,f1p,x0,eps,del,3,N)
x13,X3,Y3=Shamanskii2(f1,f1p,x0,eps,del,4,N)
x14,X4,Y4=secante(f1,x0,eps,del,N)
x15,X5,Y5=minewton(f1,f1p,x0,eps,del,N)

#Residuo de cada método para función cos(x)-x
df = DataFrame(Iteration = X1, Shamanskii_2 = Y1,Shamanskii_3 = Y2,Shamanskii_4 = Y3,Secante=Y4,Newton=Y5)
#=Ignorar valores que sea 0.0, ya que al implementar se inicializaron los vectores con ceros, por lo que luego de 
converger bajo los parametros deseados el resto de valores del vector residuo son cero por defecto =# 

Row,Iteration,Shamanskii_2,Shamanskii_3,Shamanskii_4,Secante,Newton
,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.377583,0.377583,0.377583,0.377583,0.377583
2,1.0,0.00365152,0.00047756,6.27132e-5,0.0277428,0.0271033
3,2.0,1.70507e-9,6.66134e-16,0.0,0.00163676,9.46154e-5
4,3.0,0.0,0.0,0.0,5.92196e-6,1.18098e-9
5,0.0,0.0,0.0,0.0,1.27974e-9,0.0
6,0.0,0.0,0.0,0.0,8.88178e-16,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0


In [97]:
x0=1.0
eps=1e-10
del=1e-10
N=6
x21,X1,Y1=Shamanskii2(f2,f2p,x0,eps,del,2,N)
x22,X2,Y2=Shamanskii2(f2,f2p,x0,eps,del,3,N)
x23,X3,Y3=Shamanskii2(f2,f2p,x0,eps,del,4,N)
x24,X4,Y4=secante(f2,x0,eps,del,N)
x25,X5,Y5=minewton(f2,f2p,x0,eps,del,N)

#Residuo de cada método para función arctan(x)
df = DataFrame(Iteration = X1, Shamanskii_2 = Y1,Shamanskii_3 = Y2,Shamanskii_4 = Y3,Secante=Y4,Newton=Y5)
#=Ignorar valores que sea 0.0, ya que al implementar se inicializaron los vectores con ceros, por lo que luego de 
converger bajo los parametros deseados el resto de valores del vector residuo son cero por defecto =#

Row,Iteration,Shamanskii_2,Shamanskii_3,Shamanskii_4,Secante,Newton
,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.785398,0.785398,0.785398,0.785398,0.785398
2,1.0,0.436525,0.386104,0.350592,0.512735,0.518669
3,2.0,0.0140354,0.00117973,7.57938e-5,0.0543255,0.116332
4,3.0,3.63209e-10,2.1202e-21,5.50316e-38,0.00476276,0.00106102
5,4.0,0.0,0.0,0.0,4.2797e-6,7.9631e-10
6,0.0,0.0,0.0,0.0,3.23313e-11,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0


In [98]:
x0=3.0
eps=1e-10
del=1e-10
N=6
x31,X1,Y1=Shamanskii2(f3,f3p,x0,eps,del,2,N)
x32,X2,Y2=Shamanskii2(f3,f3p,x0,eps,del,3,N)
x33,X3,Y3=Shamanskii2(f3,f3p,x0,eps,del,4,N)
x34,X4,Y4=secante(f3,x0,eps,del,N)
x35,X5,Y5=minewton(f3,f3p,x0,eps,del,N)

#Residuo de cada método para función sin(x)
df = DataFrame(Iteration = X1, Shamanskii_2 = Y1,Shamanskii_3 = Y2,Shamanskii_4 = Y3,Secante=Y4,Newton=Y5)
#=Ignorar valores que sea 0.0, ya que al implementar se inicializaron los vectores con ceros, por lo que luego de 
converger bajo los parametros deseados el resto de valores del vector residuo son cero por defecto =#

Row,Iteration,Shamanskii_2,Shamanskii_3,Shamanskii_4,Secante,Newton
,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.14112,0.14112,0.14112,0.14112,0.14112
2,1.0,9.6424e-6,9.74718e-8,9.8531e-10,0.00128079,0.000953889
3,2.0,1.22465e-16,1.22465e-16,1.22465e-16,4.25074e-6,2.89316e-10
4,0.0,0.0,0.0,0.0,1.15851e-12,1.22465e-16
5,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0


In [104]:
x0=0.5
eps=1e-10
del=1e-10
N=24
x41,X1,Y1=Shamanskii2(f4,f4p,x0,eps,del,2,N)
x42,X2,Y2=Shamanskii2(f4,f4p,x0,eps,del,3,N)
x43,X3,Y3=Shamanskii2(f4,f4p,x0,eps,del,4,N)
x44,X4,Y4=secante(f4,x0,eps,del,N)
x45,X5,Y5=minewton(f4,f4p,x0,eps,del,N)

#Residuo de cada método para función x^2
df = DataFrame(Iteration = X1, Shamanskii_2 = Y1,Shamanskii_3 = Y2,Shamanskii_4 = Y3,Secante=Y4,Newton=Y5)
#=Ignorar valores que sea 0.0, ya que al implementar se inicializaron los vectores con ceros, por lo que luego de 
converger bajo los parametros deseados el resto de valores del vector residuo son cero por defecto =#

Row,Iteration,Shamanskii_2,Shamanskii_3,Shamanskii_4,Secante,Newton
,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.25,0.25,0.25,0.25,0.25
2,1.0,0.0351562,0.0232086,0.0166759,0.0618734,0.0625
3,2.0,0.00494385,0.00215456,0.00111234,0.0275917,0.015625
4,3.0,0.000695229,0.000200017,7.4197e-5,0.00991968,0.00390625
5,4.0,9.77665e-5,1.85685e-5,4.9492e-6,0.00387682,0.000976562
6,5.0,1.37484e-5,1.7238e-6,3.30129e-7,0.00146786,0.000244141
7,6.0,1.93337e-6,1.60028e-7,2.20208e-8,0.000562556,6.10352e-5
8,7.0,2.7188e-7,1.48561e-8,1.46886e-9,0.000214602,1.52588e-5
9,8.0,3.82332e-8,1.37916e-9,9.79784e-11,8.20108e-5,3.8147e-6


In [99]:
x0=10.0
eps=1e-10
del=1e-10
N=10
x51,X1,Y1=Shamanskii2(f5,f5p,x0,eps,del,2,N)
x52,X2,Y2=Shamanskii2(f5,f5p,x0,eps,del,3,N)
x53,X3,Y3=Shamanskii2(f5,f5p,x0,eps,del,4,N)
x54,X4,Y4=secante(f5,x0,eps,del,N)
x55,X5,Y5=minewton(f5,f5p,x0,eps,del,N)

#Residuo de cada método para función x^2+1
df = DataFrame(Iteration = X1, Shamanskii_2 = Y1,Shamanskii_3 = Y2,Shamanskii_4 = Y3,Secante=Y4,Newton=Y5)
#=Ignorar valores que sea 0.0, ya que al implementar se inicializaron los vectores con ceros, por lo que luego de 
converger bajo los parametros deseados el resto de valores del vector residuo son cero por defecto =#

Row,Iteration,Shamanskii_2,Shamanskii_3,Shamanskii_4,Secante,Newton
,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,101.0,101.0,101.0,101.0,101.0
2,1.0,14.5047,9.70037,7.07436,25.2519,25.5025
3,2.0,2.37234,1.30898,1.01155,11.4501,6.63583
4,3.0,1.07766,35.9154,1.83675e20,4.34522,1.95332
5,4.0,70.9376,3.67216,1.22518e19,1.94194,1.00057
6,5.0,10.2792,1.03427,8.17237e17,1.07666,437.924
7,6.0,1.7951,2.37731e6,5.45126e16,1.34369,109.731
8,7.0,1.46653,2.20696e5,3.63618e15,15.1147,27.6852
9,8.0,2.52322,20488.5,2.42546e14,2.02016,7.18066


In [96]:
#Raices
df=DataFrame(Function=["Cos(x)-x","arctan(x)","sin(x)","x^2","(x^2)+1"],Root_Shamanskii2=[x11,x21,x31,x41,x51],Root_Shamanskii3=[x12,x22,x32,x42,x52],Root_Shamanskii4=[x13,x23,x33,x43,x53],Root_Secante=[x14,x24,x34,x44,x54],Root_Newton=[x15,x25,x35,x45,x55])

Row,Function,Root_Shamanskii2,Root_Shamanskii3,Root_Shamanskii4,Root_Secante,Root_Newton
,String,Float64,Float64,Float64,Float64,Float64
1,Cos(x)-x,0.739085,0.739085,0.739085,0.739085,0.739085
2,arctan(x),0.0,-2.1202e-21,5.50316e-38,-3.23313e-11,0.0
3,sin(x),3.14159,3.14159,3.14159,3.14159,3.14159
4,x^2,0.00139046,0.000400035,0.000148394,0.0237183,0.0078125
5,(x^2)+1,16.7081,-13.2636,-1.03883e6,-0.276886,0.0410842


De las 5 primeras tablas, en las cuales se observa el residuo de cada método según la iteración, se pueden inferir ciertos comportamientos, el primero de ellos es que el método de la secante converge comparativamente más lento que los otros 2 métodos, especiamente para $cos(x)-x$ y $sin(x)$ es apreciable que este método puede tardar el doble de iteraciones que el método de Shamanskii para sus diversos ordenes.

Asimismo, es apreciable que el método que converge comparativamente más rápido es el método de Shamanskii, especialmente el de orden 4, esto es notable especialmente para $x^2$ donde es evidente que el método de Shamanskii está convergiendo más rápido a mayor es el orden.

También es importante notar como el método de Newton converge comparativamente más rápido que el de la secante, pero más lento que los de Shamanskii; lo cual es una evidencia de la importancia del cálculo de la derivada en un método numérico, se puede pensar que al aproximar la derivada (como en el método de la secante) se deben realizar más iteraciones para alcanzar un mejor resultado que usando la derivada explicitamente, y asimismo se observa como el algoritmo introducido por Shamanskii mejora el método de Newton, realizando una cantidad de cálculos superior según el orden pero computando la derivada una menor cantidad de vecez.

Por último, en la tabla 6 se observa como para estos casos los métodos convergieron a la misma raíz (en caso de existir), aparantemene se obtienen buenos resultados para las funciones trigonométricas para todos los métodos, sin embargo para $x^2$ los restultados no son los mejores, siendo los mejores Shamanskii de ordenes 3 y 4 obteniendo 3 ceros después del punto decimal y el peor el de la secante con tan solo un cero, notese que los métodos que menos iteraciones hicieron fueron los que mejor exactitud tuvieron. Asimismo para $x^2+1$ que no tiene raices reales se puede ver como los métodos divergen y en la tabla 5 es observable como saltan de punto a punto sin acercarse a ningún resultado, lo cual es bueno ya que otorga una forma de controlar que se esten haciendo bien las cosas. 

## Problema 3
 Considere un plano cuya pendiente varía con tasa constante $\omega$ y un punto de masa quieto en el tiempo $t=0$. En el tiempo $t>0$ su posición es dada por
$$s(t,\omega)= \frac{g}{2\omega^2}\Big[ \sinh(\omega t)-\sin(\omega t) \Big] $$
donde $g=9.8 \frac{m}{s^2}$. Suponga que el objeto se ha movido 1 metro en un segundo, calcule el valor correspondiente de $\omega$ con 12 decimales exactos. 

#### Solución

Del problema, se tiene que:

$$s(1,\omega)= \frac{g}{2\omega^2}\Big[ \sinh(\omega t)-\sin(\omega t) \Big]=1 $$

Es decir

$$f(\omega)=\frac{g}{2\omega^2}\Big[ \sinh(\omega)-\sin(\omega) \Big]-1=0 $$

Al derivar se obtiene que

$$ f'(\omega)=\frac{g}{2}\frac{\omega(\cosh(\omega)-\cos(\omega))-2(\sinh(\omega)-\sin(\omega))}{\omega^3}$$

Al resolver usando el método de Shamanskii de orden 4 implementado anteriormente, con $\omega_0=0.5$, con el fin de evitar pérdida significativa de bits en la resta $\sinh(\omega)-\sin(\omega)$

In [148]:
function f_3(x)
    return (9.8/(2*x^2))*(sinh(x)-sin(x))-1
end
function fp_3(x)
    return (9.8/2)*(x*(cosh(x)-cos(x))-2*(sinh(x)-sin(x)))/(x^3)
end
Shamanskii(f_3,fp_3,0.5,1e-12,1e-12,4,15)

residuo f(x(1,4))=-2.55351295663786e-15


0.6121425707030923

Para evitar la pérdida significativa de bits también se pudo haber usado la expansión en serie de Taylor del $\sinh(\omega)$ y $\sin(\omega)$, del cual se obtiene que
$$ f(\omega)=g\sum_{n=0}^{\infty}\frac{x^{4n+1}}{(4n+3)!} \text{,}$$
y al derivar esta última expresión
$$ f'(\omega)=g\sum_{n=0}^{\infty}\frac{(4n+1)x^{4n}}{(4n+3)!} \text{,}$$
al realizar el truncamiento para calcular estas funciones con errores menores a $10^-12$ se obtienen que
$$|f(\omega)-P(\omega)| < 10^{-12} \text{,}$$
es decir
$$\Big|\frac{g\xi^{4n+1}}{(4n+3)!} \Big| < 10^{-12} \text{,}$$
se puede considerar que $|\xi|<1$, por lo que al resolver
$$\Big|\frac{g}{(4n+3)!} \Big| < 10^{-12} \text{,}$$
se obtiene un resultado de $n=4$.
Al realizar el proceso análogo a la derivada también se obtuvo $n=4$.
Por lo que al implementar en el código estas funciones, utilizando el esquema de Horner, se obtiene el siguiente resultado.

In [147]:
function h(x)
    return 9.8*((x/6)*(1+(x^4/(4*5*6*7))*(1+(x^4/(8*9*10*11))*(1+(x^4/(12*13*14*15))*(1+(x^4/(16*17*18*19)))))))-1
end
function hp(x)
    return 9.8*((1/6)+(x^4/(2*3*4*5*6*7))*(5+(x^4/(8*9*10*11))*(9+(x^4/(12*13*14*15))*(13+(x^4/(12*18*19))))))
end
Shamanskii(h,hp,0.5,1e-12,1e-12,4,15)

residuo f(x(1,4))=-3.885780586188048e-15


0.612142570703092

In [149]:
using Roots

In [150]:
find_zero(f_3,(0.5,0.7))

0.6121425707030941

Del problema planteado se obtuvieron 3 resultados, 2 a través del método de Shamanskii, el primero usando la función explicitamente como fue dada en el problema, el segundo usando la expansión en serie de Taylor de la función seno y seno hipérbolica y aplicando el sequema de Horner, y la tercera a través de la libreria Roots. Se puede ver que los valores de los 3 casos difieren hasta llegado el décimo tercer (13) digito después del punto décimal. Se puede notar que la pérdida significativa de bits no parece haber afectado el primer caso, probablemente débido a que el valor inicial está lo suficientemente lejano del cero para tener este problema.

## Problema 4
La longitud maxima de una varilla que se puede arrastrar de un extremo a otro deslizandola por un pasillo como el de la fiugra es 
$$L=\frac{l_2}{\sin(\pi-\gamma-\alpha)}+\frac{l_1}{\sin(\alpha)}$$
donde $\alpha$ es la solución de la ecuación
$$ l_2 \frac{\cos(\pi-\gamma-\alpha)}{\sin^2(\pi-\gamma-\alpha)}-l_1\frac{\cos(\alpha)}{\sin^2(\alpha)}=0.$$
Calcule $\alpha$ cuando $l_2=10$, $l_1=8$ y $\gamma=\frac{3\pi}{5}$. ¿Cuántos decimales puede garantizar para $\alpha$ y $L$?

### Solución

Al reemplazar los valores dados en la ecuación se obtiene que
$$  10 \frac{\cos\Big(\frac{2\pi}{5}-\alpha\Big)}{\sin^2\Big(\frac{2\pi}{5}-\alpha\Big)}-8\frac{\cos(\alpha)}{\sin^2(\alpha)}=0. $$

Al implementarlo en el método de Shamanskii de orden 4

In [161]:
function g(x)
    return 10*(cos((2*pi/5)-x)/(sin((2*pi/5)-x))^2)-8*(cos(x)/(sin(x))^2)
end
function gp_aux(x)
    return 10*(csc((2*pi/5)-x)^4)*(sin((2*pi/5)-x)^3+sin(2((2*pi/5)-x))*cos((2*pi/5)-x))
end
function gp_aux2(x)
    return -8*(csc(x)^4)*(sin(x)^3+sin(2x)*cos(x))
end
function gp(x)
    return gp_aux(x)-gp_aux2(x)
end

gp (generic function with 1 method)

In [164]:
Shamanskii(g,gp,0.5,1e-12,1e-12,4,15)

residuo f(x(2,4))=1.0658141036401503e-14


0.5962799274654735

Por lo que concluimos que $\alpha=0.5962799274654735$.

## Problema 5 
Seleccione alguno de los métodos implementados en julia para resolver ecuaciones o sistemas de ecuaciones. Describa el método implementado con el mayor detalle posible. 

Si usa Python selecciones dos de los métodos en https://docs.scipy.org/doc/scipy-0.13.0/reference/optimize.html en la sección Root finding, diga cual es el método numerico implementado (e.g, similar a newton, usa derivadas, no usa derivadas, combina varios métodos, llama alguna otra subrutina conocida encontrada en netlib, cuales son las toleracias por defecto, etc)  y para cada uno de los métodos seleccionados implementar un ejemplo numérico con una ecuación escalar o una ecuación vectorial según sea el caso. Imprima la solución calculada asi como alguna informacion adicional (residuo, cantidad de iteraciones, etc). 

### Solución

Consideremos los métodos implemetados dentreo de *fzero* en Julia. 

Para *fzero* el método númerico depende del orden de convergencia deseado, por defecto se selecciona el orden 1, implementado a través del método de bisección, el orden 1 usa el método de la secante, el método de orden 2 usa el método de Steffensen; en general usa métodos que no depende de la derivada de la función, como los mencionados antes, a parte de esos también existen métodos de orden 5, 8 y 16 libres del uso de derivadas. Por default la tolerancia es la misma que la precisión de máquina.

El orden 5 usa el algoritmo mencionado en "A New Fifth Order Derivative Free Newton-Type Method for Solving Nonlinear
Equations" de Manoj Kumar, el cual realiza 4 llamadas de la función por paso, el método se define como 
$$ y_n=x_n-\frac{f(x_n)}{f[w_n,x_n]} \text{,}$$  
$$ z_{n+1}=x_n-\frac{f(x_n+f(y_n))}{f[w_n,x_n]} \text{,}$$  
$$ x_{n+1}=z_{n+1}-\frac{f(z_n+1)\cdot f[w_n,x_n]}{f[x_n,y_n]f[w_n,y_n]} \text{,}$$  
donde $f[x_n,w_n]=\frac{f(w_n)-f(x_n)}{f(x_n)}$ y $w_n=x_n+f(x_n)$

Para el orden 8 usa el método de Thukral mostrado en "New Eighth-Order Derivative-Free Methods for Solving Nonlinear Equations" por Rajinder Thukral, el método se define como

$$ w_n=x_n+\beta_i^{-1}f(x_n) \text{,}$$
$$ y_n=x_n-\frac{f(x_n)^2}{f(w_n)-f(x_n)} \text{,}$$
$$ z_{n}=y_n-\phi_j \Big(\frac{x_n-y_n}{f(x_n)-f(y_n)} \Big)f(y_n) \text{,}$$
$$ x_{n+1}=z_{n}-\omega_{k}\xi_l\Big(\frac{f(z_n)-f(y_n)}{z_n-y_n}-\frac{f(y_n)-f(x_n)}{y_n-x_n}+\frac{f(z_n)-f(x_n)}{z_n-x_n} \Big)^{-1}f(z_n) \text{,}$$
si $i\in\mathbb{R^{+}}$, los parámetros $\beta_i$,$\phi_j$,$\xi_l$ y $\omega_k$ están dados por
$$\beta_{i}=i^{-1} \text{,}$$
$$\phi_1=\Big(1-\frac{f(y_n)}{f(w_n)} \Big)^{-1} \text{ , }\phi_2=\Big(1+\frac{f(y_n)}{f(w_n)} \Big) \text{,}$$
$$\omega_1=\Big(1-\frac{f(z_n)}{f(w_n)} \Big)^{-1} \text{ , }\omega_2=\Big(1+\frac{f(z_n)}{f(w_n)}+\Big(\frac{f(z_n)}{f(w_n)} \Big)^2 \Big) \text{,}$$
$$\xi_1=\Big(1-\frac{2 f(y_n)^3}{f(w_n)^2 f(x_n)} \Big)^{-1} \text{ , }\xi_2=\Big(1+\frac{2 f(y_n)^3}{f(w_n)^2 f(x_n)} \Big)^{-1} \text{.}$$
El artículo que comenta este método menciona que a partir de variaciones de los parámetros antes colocados se pueden generar múltiples métdoos de octavo orden, sin embargo, al revisar la documentación de la libreria Roots es sabido que el método usado corresponde a uno de aquellos, mas no es clara la escogencia de estos parámetros.

Ahora veamos una implementació de estos métodos

In [167]:
function U(x)
    return asin(x^2-1)-x/2+1
end

U (generic function with 1 method)

In [184]:
print("Raices \n")
print("Orden 0: ",fzero(U,0.1,order=0),"\n","Orden 1: ",fzero(U,0.1,order=1),"\n")
print("Orden 2: ",fzero(U,0.1,order=2),"\n","Orden 5: ",fzero(U,0.1,order=5),"\n")
print("Orden 8: ",fzero(U,0.1,order=8),"\n","Orden 16: ",fzero(U,0.1,order=16),"\n")

print("Residuos \n")
print("Orden 0: ",U(fzero(U,0.1,order=0)),"\n","Orden 1: ",U(fzero(U,0.1,order=1)),"\n")
print("Orden 2: ",U(fzero(U,0.1,order=2)),"\n","Orden 5: ",U(fzero(U,0.1,order=5)),"\n")
print("Orden 8: ",U(fzero(U,0.1,order=8)),"\n","Orden 16: ",U(fzero(U,0.1,order=16)),"\n")

Raices 
Orden 0: 0.5948109683983691
Orden 1: 0.5948109683983692
Orden 2: 0.5948109683983691
Orden 5: 0.5948109683983689
Orden 8: 0.5948109683983692
Orden 16: 0.5948109683983692
Residuos 
Orden 0: 0.0
Orden 1: 0.0
Orden 2: 0.0
Orden 5: -4.440892098500626e-16
Orden 8: 0.0
Orden 16: 0.0


De los 2 métodos vistos se puede observar que aparecen expresiones similares a la de Newton, esto es débido a que como lo mencionan sus respectivos artículos se parte de una base común que es la expresión de Newton y a partir de esta se pueden realizar diversas manipulaciones, definir nuevos parámetros y variables, aproximar la derivada, con el fin de obener un resultado preciso pero sin calcular la derivada.

Asimismo, se puede observar que las raices de la función trascendental calculadas coinciden hasta el décimo cuarto (14) digito decimal, por ello mismo los residuos pueden diferir, y aunque analíticamente seguramente lo hagan al calcularlos computacionalmente son más pequeños que la precisión de máquina, por lo que los vemos como un cero. Aún así, es notable que el orden 5 queda con un residuo del orden de $10^{-16}$, haciendo más explicita esta diferencia comentada previamente. 

#### Fuentes
 A New Fifth Order Derivative Free Newton-Type Method for Solving Nonlinear
  Equations by Manoj Kumar, Akhilesh Kumar Singh, and Akanksha, Appl. Math. Inf. Sci. 9, No. 3, 1507-1513 (2015), DOI:
  10.12785/amis/090346 (https://doi.org/10.12785/amis/090346).
  
 New Eighth-Order Derivative-Free Methods for Solving Nonlinear Equations
  by Rajinder Thukral, International Journal of Mathematics and Mathematical Sciences Volume 2012 (2012), Article ID
  493456, 12 pages DOI: 10.1155/2012/493456 (https://doi.org/10.1155/2012/493456).
  
 https://github.com/JuliaMath/Roots.jl